In [1]:
import numpy as np
from datasets import load_dataset
from transformers import AutoImageProcessor, AutoModel

tiny_imagenet = load_dataset('zh-plus/tiny-imagenet', split='valid')
processor = AutoImageProcessor.from_pretrained('facebook/dinov2-small')
model = AutoModel.from_pretrained('facebook/dinov2-small')

/home/czaloom/valor/.env-velour/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def embed(x):
    inputs = processor(images=x, return_tensors="pt")
    outputs = model(**inputs)
    retval = (
        outputs
        .last_hidden_state
        .flatten()
        .detach()
        .numpy()
    )
    retval = retval[np.newaxis, :]
    return retval

In [3]:
classes = {
    0: "fish",
    1: "ground_spider",
    2: "frog",
    5: "snake",
    8: "web_spider",
    19: "penguin"
}
id_labels = classes.keys()
text_labels = list(classes.values())

In [4]:
embeddings = [
    [
        embed(image['image'])
        for image in tiny_imagenet 
        if image['label'] == label
    ]
    for label in id_labels
]

In [ ]:
from collections import defaultdict
from dataclasses import dataclass
from typing import Callable

import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist
from scipy.stats import cramervonmises_2samp, ks_2samp

@dataclass
class EmbeddingMatrix:
    """
    Maps scores from each query label to each reference label.
    """

    statistics: dict[str, dict[str, float]]
    pvalues: dict[str, dict[str, float]]

In [ ]:
def get_inner_product_function(dataset: np.ndarray):
    return lambda x: np.dot(dataset, x)


def get_projections(
    basis: np.ndarray,
    inner_product: Callable[[np.ndarray], np.ndarray],
) -> np.ndarray:
    """
    Computes orthogonal projection of a dataset onto a plane with orthonormal basis `basis`.
    The dataset is represented abstractly by the Callable `inner_product`, which is a function
    that takes in a vector `v` and returns the array of inner products <x, v> for all x in the
    dataset.

    Parameters
    ----------
    basis: np.ndarray
        Should be k x d, all length 1, and orthogonal
    inner_product: Callable[np.ndarray, np.ndarray]

    Returns
    -------
    projections: np.ndarray
        N x k
    """
    # check that basis is orthonormal
    dots = np.einsum("ik,jk->ij", basis, basis)
    np.testing.assert_almost_equal(dots, np.identity(dots.shape[0]))

    res = []
    for x in basis:
        res.append(inner_product(x))
    res = np.array(res)
    return res.T

In [ ]:
def _compute_metrics_from_projections(
    queries: list[list[np.ndarray]],
    references: list[list[np.ndarray]] | None,
    classes: list[str],
    method: str,
    func: Callable,
) -> EmbeddingMatrix:
    """
    Computes metrics using a selectable scipy.stats function.

    Parameters
    ----------
    queries : list[list[np.ndarray]]
        A list of query embedding lists.
    references : list[list[np.ndarray]], optional
        A list of reference embedding lists.
    classes : list[str]
        A list of labels mapped to the distance lists.
    method : str
        The method of calculating distance.
    func : Callable
        A scipy.stats function.

    Returns
    -------
    EmbeddingMatrix
        A metric containing a confusion matrix for both p-value and distance metric.
    """
    pvalues = defaultdict(dict)
    statistics = defaultdict(dict)

    def cast_and_round(x):
        x = float(x)
        return round(x, 3)

    queries_are_references = references is None
    references = queries

    vector = np.random.normal(size=(1, references[0][0].shape[1]))
    vector = vector / np.linalg.norm(vector)

    for i, query in enumerate(queries):
        for j, reference in enumerate(references):
            if queries_are_references and i == j:
                # split the set in two and measure how similarly distributed it is.
                split_idx = len(reference) // 2
                reference_proj = get_projections(
                    vector,
                    get_inner_product_function(
                        np.concatenate(reference[:split_idx])
                    )
                )
                query_proj = get_projections(
                    vector,
                    get_inner_product_function(
                        np.concatenate(query[split_idx:])
                    )
                )
            else:
                reference_proj = get_projections(
                    vector,
                    get_inner_product_function(
                        np.concatenate(reference)
                    )
                )
                query_proj = get_projections(
                    vector,
                    get_inner_product_function(
                        np.concatenate(query)
                    )
                )
            metric = func(reference_proj, query_proj)

            label_i = classes[i]
            label_j = classes[j]
            pvalues[label_i][label_j] = cast_and_round(metric.pvalue)
            statistics[label_i][label_j] = cast_and_round(metric.statistic)

    return EmbeddingMatrix(pvalues=pvalues, statistics=statistics)

In [ ]:
def compute_cvm(
    queries: list[list[np.ndarray]],
    classes: list[str],
    references: list[list[np.ndarray]] | None = None,
    method: str = "cosine",
) -> EmbeddingMatrix:
    return _compute_metrics_from_projections(
        queries=queries,
        references=references,
        classes=classes,
        method=method,
        func=cramervonmises_2samp,
    )


def compute_ks(
    queries: list[list[np.ndarray]],
    classes: list[str],
    references: list[list[np.ndarray]] | None = None,
    method: str = "cosine",
) -> EmbeddingMatrix:
    return _compute_metrics_from_projections(
        queries=queries,
        references=references,
        classes=classes,
        method=method,
        func=ks_2samp,
    )

Compute metrics

In [5]:
from embedding_metrics import compute_cvm, compute_ks, create_dataframe

In [6]:
cvm = compute_cvm(embeddings, text_labels)
ks = compute_ks(embeddings, text_labels)

Create pandas dataframes

In [7]:
cvm_statistics, cvm_pvalues = create_dataframe(cvm, text_labels)
ks_statistics, ks_pvalues = create_dataframe(ks, text_labels)

In [8]:
cvm_statistics

Reference                                               
                         fish ground_spider   frog  snake web_spider penguin
Query fish              0.060         0.458  0.141  0.030      0.648   0.848
      ground_spider     0.458         0.068  0.260  0.435      0.156   0.220
      frog              0.141         0.260  0.048  0.117      0.261   0.385
      snake             0.030         0.435  0.117  0.031      0.647   0.830
      web_spider        0.648         0.156  0.261  0.647      0.068   0.031
      penguin           0.848         0.220  0.385  0.830      0.031   0.116

In [9]:
cvm_pvalues

Reference                                               
                         fish ground_spider   frog  snake web_spider penguin
Query fish              0.845         0.051  0.424  0.984      0.017   0.006
      ground_spider     0.051         0.794  0.178  0.058      0.376   0.234
      frog              0.424         0.178  0.913  0.514      0.177   0.079
      snake             0.984         0.058  0.514  0.987      0.017   0.006
      web_spider        0.017         0.376  0.177  0.017      0.794   0.980
      penguin           0.006         0.234  0.079  0.006      0.980   0.531

In [10]:
ks_statistics

Reference                                             
                         fish ground_spider  frog snake web_spider penguin
Query fish               0.28          0.28  0.22  0.24       0.14    0.24
      ground_spider      0.28          0.16  0.36  0.32       0.22    0.48
      frog               0.22          0.36  0.28  0.14       0.26    0.20
      snake              0.24          0.32  0.14  0.20       0.16    0.18
      web_spider         0.14          0.22  0.26  0.16       0.24    0.28
      penguin            0.24          0.48  0.20  0.18       0.28    0.16

In [11]:
ks_pvalues

Reference                                               
                         fish ground_spider   frog  snake web_spider penguin
Query fish              0.285         0.039  0.179  0.112      0.717   0.112
      ground_spider     0.039         0.915  0.003  0.012      0.179   0.000
      frog              0.179         0.003  0.285  0.717      0.068   0.272
      snake             0.112         0.012  0.717  0.710      0.549   0.396
      web_spider        0.717         0.179  0.068  0.549      0.475   0.039
      penguin           0.112         0.000  0.272  0.396      0.039   0.915